# Welcome to your Quantitative Social Sciences analysis toolkit!

In this notebook, you will perform the following operations:
    - load data
    - make histograms of variables of interest
    - calculate correlations between variables of interest
    - calculate group differences on variables of interest

In [1]:
import pandas as pd

df = pd.read_csv('data/anes_pilot_2018.csv', low_memory=False)

In [97]:
%matplotlib inline 

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from matplotlib import pyplot as plt

@interact(x=df.columns)
def categorical_plot(x='gender'):
    plt.figure(figsize=(10,5))
    ax=plt.gca()
    return df.groupby(x).size().plot(ax=ax, kind='bar')

pass

interactive(children=(Dropdown(description='x', index=284, options=('version', 'caseid', 'weight', 'weight_sps…

In [98]:
%matplotlib inline 

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def cast(s):
    try:
        return float(s)
    except:
        return None

@interact(dependent=df.columns, independent=df.columns)
def comparison_plot(dependent='gehope', independent='gender', bins=list(range(10,100,5))):
    plt.figure(figsize=(10,5))
    ax = plt.gca()
    
    print(len(df[dependent]), len(df[dependent].dropna()))

    dff = df.copy()
    dff = dff.dropna(subset=[dependent, independent])
    
    if len(dff[independent].unique()) > 15:
        plt.scatter(dff[independent], dff[dependent])
        return ax
    
    try:
        dff.groupby(independent)[dependent].plot(ax=ax, kind='hist', alpha=.33, bins=bins)
    except:
        dff[dependent] = df[dependent].apply(cast)
        dff.groupby(independent)[dependent].plot(ax=ax, kind='hist', alpha=.33, bins=bins)
        
    ax.legend()
    return ax


interactive(children=(Dropdown(description='dependent', index=59, options=('version', 'caseid', 'weight', 'wei…

In [9]:
import statsmodels.api as sm


def cast(s):
    try:
        return float(s)
    except:
        return None


@interact(dependent=df.columns, independent=df.columns)
def comparison_plot(dependent='gehope', independent='gender'):
    
    print(len(df[dependent]), len(df[dependent].dropna()))

    dff = df.copy()
    dff = dff.dropna(subset=[dependent, independent])
    try:
        dff[dependent] = dff[dependent].apply(cast)
        results = sm.OLS(dff[dependent], dff[independent]).fit()
        print(results.summary())
        
        return pd.crosstab(df[independent], df[dependent])
    
    except:
        return 'comparison does not run'


interactive(children=(Dropdown(description='dependent', index=59, options=('version', 'caseid', 'weight', 'wei…